In [9]:
from caveclient import CAVEclient
from datetime import datetime

cave_client_public = CAVEclient('minnie65_public_v117')
cave_client = CAVEclient('minnie65_phase3_v1')

# get timestamp for public release
past_timestamp = cave_client.materialize.get_timestamp(117)

# get old nuclei classification table
nuclei_before = cave_client_public.materialize.query_table('nucleus_neuron_svm')

# get new nuclei classification table
nuclei_now = cave_client.materialize.query_table('allen_neuron_nonneuron_svm_v0',
                                                timestamp=datetime.utcnow())


nuclei_table = nuclei_before[['id', 'pt_root_id', 'pt_position']].merge(nuclei_now[['id', 'cell_type']], on='id', how='right')

neurons = nuclei_table[nuclei_table.cell_type == 'neuron']

05/05/2022 05:04:03 PM INFO: uniq_root_ids [864691131942873927 864691132631239154 864691132640888901 ...
 864691137197409089 864691137197412673 864691137197413441] 
05/05/2022 05:04:06 PM INFO: pt_supervoxel_id has 35 to update 
05/05/2022 05:04:06 PM INFO: num zero svids: 0 
05/05/2022 05:04:06 PM INFO: all_svids dtype int64 
05/05/2022 05:04:06 PM INFO: all_svid_lengths [35] 


In [21]:
queue_url = sqs.get_or_create_queue("Root_ids")
running_list = []
for i, root_id in enumerate(neurons.pt_root_id.unique()[:100]):
    running_list.append({
            'Id': str(i),
            'MessageBody': str(root_id),
        })
    if i % 10 == 0:
        sqs.send_batch(queue_url, running_list)
        running_list = []

05/05/2022 05:19:28 PM INFO: Found credentials in shared credentials file: ~/.aws/credentials 
05/05/2022 05:19:29 PM INFO: Found credentials in shared credentials file: ~/.aws/credentials 
05/05/2022 05:19:29 PM INFO: Found credentials in shared credentials file: ~/.aws/credentials 
05/05/2022 05:19:29 PM INFO: Found credentials in shared credentials file: ~/.aws/credentials 
05/05/2022 05:19:29 PM INFO: Found credentials in shared credentials file: ~/.aws/credentials 
05/05/2022 05:19:30 PM INFO: Found credentials in shared credentials file: ~/.aws/credentials 
05/05/2022 05:19:30 PM INFO: Found credentials in shared credentials file: ~/.aws/credentials 
05/05/2022 05:19:30 PM INFO: Found credentials in shared credentials file: ~/.aws/credentials 
05/05/2022 05:19:30 PM INFO: Found credentials in shared credentials file: ~/.aws/credentials 
05/05/2022 05:19:31 PM INFO: Found credentials in shared credentials file: ~/.aws/credentials 
05/05/2022 05:19:31 PM INFO: Found credentials in 